 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P TSX 60 Index.

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import seaborn as sns

%matplotlib inline


# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp_tsx_history.csv`: Contains historical closing prices of the S&P TSX 60 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data.

In [ ]:
whales_data = Path("Resources/whale_returns.csv")

In [ ]:
# Reading whale returns
whales_df = pd.read_csv(whales_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort whale returns DataFrame by Date Index
whales_df.sort_index(inplace=True)
whales_df.head()

In [ ]:
# Count nulls
whales_df.isnull().sum()

In [ ]:
# Drop nulls
# Used the `dropna` function to drop whole records that have at least one null value
whales_df.dropna(inplace=True)
# Check no more nulls
whales_df.isnull().sum()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data.

In [ ]:
# Reading algorithmic returns
algo_data = Path("Resources/algo_returns.csv")
algo_df = pd.read_csv(algo_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort algorithmic returns DateFrame by Date Index
algo_df.sort_index(inplace=True)
algo_df.head()

In [ ]:
# Count nulls
algo_df.isnull().sum()

In [ ]:
# Drop nulls
algo_df.dropna(inplace=True)
##algo_df.isnull().sum()

## S&P TSX 60 Returns

Read the S&P TSX 60 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P TSX 60 Closing Prices
sp60_data = Path("Resources/sp_tsx_history.csv")
sp60_df = pd.read_csv(sp60_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort S&P TSX 60 DataFrame by Date Index
sp60_df.sort_index(inplace=True)
sp60_df.head()

In [ ]:
# Check Data Types
print(sp60_df.dtypes)

In [ ]:
# Fix Data Types

# Remove $ sign and , from sp60 Close values 
sp60_df['Close'] = sp60_df['Close'].str.replace('$', '')
sp60_df['Close'] = sp60_df['Close'].str.replace(',', '')

# Convert sp60 Close values to type float
sp60_df['Close'] = sp60_df['Close'].astype(float)

##print (sp60_df['Close'])
print(sp60_df.dtypes)


In [ ]:
# Calculate Daily Returns
sp60_df['Daily Returns'] = sp60_df['Close'].pct_change()
##sp60_df.head()

In [ ]:
# Drop nulls
##sp60_df.isnull().sum()
sp60_df.dropna(inplace=True)

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
# Renamed 'Daily Returns' Column to S&P TSX 
columns = ["S&P Close", "S&P TSX"]
sp60_df.columns = columns
sp60_df.head()

## Combine Whale, Algorithmic, and S&P TSX 60 Returns

In [ ]:
# Drop Column with closing prices, so we have only daily returns for 
# S&P TSX in the sp60_df
sp60_df = sp60_df.drop(columns=["S&P Close"], errors ="ignore")
sp60_df.head()

# Join Whale Returns, Algorithmic Returns, and the S&P TSX 60 Returns into a 
# single DataFrame with columns for each portfolio's returns.
df_daily_returns = pd.concat([whales_df, algo_df, sp60_df], axis="columns", join="inner")
df_daily_returns

## Note for self can check index using .index
## df_daily_returns.index

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
df_daily_returns.plot(figsize=(20,10), title="Daily Returns of Whales, Algos and S&P TSX")

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_returns = (1 + df_daily_returns).cumprod() - 1
cumulative_returns

# Plot cumulative returns
cumulative_returns.plot(figsize=(20,10), title="Cumulative Returns of Whales, Algos and S&P TSX")

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios.
4. Determine which portfolios are riskier than the S&P TSX 60.
5. Calculate the Annualized Standard Deviation.

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
df_daily_returns.plot.box(figsize=(20,10), title="Box Plot for Whales, Algos and S&P TSX")

##Note for self kept in comments for future reference
##sns.set_style("white")
##sns.set(rc={"figure.figsize":(20, 10)})
##sns.boxplot(x = None, y = None, hue = None, data=combined_returns)

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
df_daily_returns.std()


### Determine which portfolios are riskier than the S&P TSX 60

In [ ]:
# Calculate the daily standard deviation of S&P TSX 60
risk_sNp = df_daily_returns['S&P TSX'].std()
##print(risk_sNp)

# Determine which portfolios are riskier than the S&P TSX 60
df_daily_returns.std() > risk_sNp

# Answer: Analysis Results show that all portfolios other than PAULSON & CO.INC. are
# riskier than S&P 60 TSX

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
volatility = df_daily_returns.std() * np.sqrt(252)
volatility

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for for all portfolios using a 21-day window.
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P TSX 60.
3. Choose one portfolio, then calculate and plot the 60-day rolling beta for it and the S&P TSX 60.

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
rolling21_std = df_daily_returns.rolling(window=21).std()

# Plot the rolling standard deviation
rolling21_std.plot(figsize=(20,10), title="21-day rolling Std Deviations")


### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
df_correlation = df_daily_returns.corr()

# Display de correlation matrix
df_correlation

# Plot the correlation
sns.heatmap(df_correlation, vmin=-1, vmax=1)


### Calculate and Plot Beta for a chosen portfolio and the S&P 60 TSX

In [ ]:
# Calculate covariance of a single portfolio. Covariance is calculated
# for BERKSHIRE HATHAWAY INC against S&P 60 TSX for rolling window of 60 days

covariance = df_daily_returns['BERKSHIRE HATHAWAY INC'].rolling(window=60).cov(df_daily_returns['S&P TSX'])
#covariance.head(60)

# Calculate variance of S&P TSX
variance = df_daily_returns['S&P TSX'].rolling(window=60).var()
##variance.head(65)

# Computing beta
berkshire_beta = covariance / variance
##berkshire_beta

# Plot beta trend
berkshire_beta.plot(figsize=(20,10), title="Rolling 60-Day Beta of BERKSHIRE HATHAWAY INC")



## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half-life.

In [ ]:
# Use `ewm` to calculate the rolling window
##df_daily_returns_ewm = df_daily_returns.ewm(halflife=21).mean() 
##df_daily_returns_ewm

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
sharpe_ratios = (df_daily_returns.mean() * 252) / (df_daily_returns.std() * np.sqrt(252))
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot.bar(figsize=(20,10), title="Sharpe Ratios")

### Determine whether the algorithmic strategies outperform both the market (S&P TSX 60) and the whales portfolios.

Write your answer here! 
Algorithmic Strategies consist of Algo 1 and Algo 2.
Based on the Sharpe Ratios - Algo 1 outperfromed the S&P TSX 60 and all the Whales. Algo 2 also outperforms S&P TSX 60 and 3 out of the 4 from Whales (Berkshire Hathaway in Whales is the only that Algo 1 did not outperform but came close behind).

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P TSX 60. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock.
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns.
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others.
5. Include correlation analysis to determine which stocks (if any) are correlated.

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Reading data from 1st stock
msft_data = Path("Resources/msft_historical.csv")
msft_df = pd.read_csv(msft_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort msft DataFrame by Date Index
msft_df.sort_index(inplace=True)
msft_df


In [ ]:
# Reading data from 2nd stock
aapl_data = Path("Resources/aapl_historical.csv")
aapl_df = pd.read_csv(aapl_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort aapl DataFrame by Date Index
aapl_df.sort_index(inplace=True)
aapl_df

In [ ]:
# Reading data from 3rd stock
amzn_data = Path("Resources/amzn_historical.csv")
amzn_df = pd.read_csv(amzn_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort L DataFrame by Date Index
amzn_df.sort_index(inplace=True)
amzn_df

In [ ]:
# Combine all stocks in a single DataFrame

# Prepare the dataframes before concatenating
# Rename 'Symbol' & `Close` Column to be specific to portfolio.
columns_msft = ["MSFT Symbol", "MSFT Close"]
msft_df.columns = columns_msft
##msft_df.head()

columns_aapl = ["AAPL Symbol", "AAPL Close"]
aapl_df.columns = columns_aapl
##aapl__df.head()

columns_amzn = ["AMZN Symbol", "AMZN Close"]
amzn_df.columns = columns_amzn
##amzn__df.head()

# Drop Symbols Column, so we have only closing columns for all 3 dataframes 
msft_df = msft_df.drop(columns=["MSFT Symbol"], errors ="ignore")
aapl_df = aapl_df.drop(columns=["AAPL Symbol"], errors ="ignore")
amzn_df = amzn_df.drop(columns=["AMZN Symbol"], errors ="ignore")

# Concatenate the 3 stock dataframes into one portfolio df
df_portfolio = pd.concat([msft_df, aapl_df, amzn_df], axis="columns", join="inner")
df_portfolio


In [ ]:
# Reset Date index
df_portfolio = df_portfolio.reset_index()
df_portfolio

In [ ]:
# Reorganize portfolio data by having a column per symbol
# Renamed 'Close' Columns to be symbols 
columns = ["Date", "MSFT", "AAPL", "AMZN"]
df_portfolio.columns = columns

# Set Index to Date and Sort
df_portfolio.set_index("Date", drop=True, inplace=True)
df_portfolio.sort_index(inplace=True)

df_portfolio

In [ ]:
# Calculate daily returns
df_portfolio_returns = df_portfolio.pct_change()
df_portfolio_returns

# Count nulls
df_portfolio_returns.isnull().sum()

# Drop NAs
df_portfolio_returns.dropna(inplace=True)
##df_portfolio_returns.isnull().sum()

# Display sample data
df_portfolio_returns.head()


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return
df_portfolio_weighted = df_portfolio_returns.dot(weights)

# Display sample data
df_portfolio_weighted



## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame

df_all_returns = pd.concat([df_daily_returns, df_portfolio_weighted], axis="columns", join="inner")
df_all_returns

# Rename Columns
columns_portfolio = ["SOROS FUND MANAGEMENT LLC", "PAULSON & CO.INC.", "TIGER GLOBAL MANAGEMENT LLC",
                     "BERKSHIRE HATHAWAY INC", "Algo 1", "Algo 2", "S&P TSX", "Custom"]
df_all_returns.columns = columns_portfolio

df_all_returns

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
df_all_returns.isnull().sum()
df_all_returns.dropna(inplace=True)

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`
volatility = df_all_returns.std() * np.sqrt(252)
volatility


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation
# for all portfolios for 21-day rolling window
all_rolling21_std = df_all_returns.rolling(window=21).std()

# Plot the rolling standard deviation
all_rolling21_std.plot(figsize=(20,10), title="21-day rolling Std Deviations")



### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation
df_all_correlation = df_all_returns.corr()
sns.heatmap(df_all_correlation, vmin=-1, vmax=1)

### Calculate and Plot the 60-day Rolling Beta for Your Portfolio compared to the S&P 60 TSX

In [ ]:
# Calculate and plot Beta
covariance_custom = df_all_returns['Custom'].rolling(window=60).cov(df_all_returns['S&P TSX'])

# Calculate variance of S&P TSX
variance_custom = df_all_returns['S&P TSX'].rolling(window=60).var()

# Computing beta
custom_beta = covariance_custom / variance_custom

# Plot beta trend
custom_beta.plot(figsize=(20,10), title="Rolling 60-Day Beta of CUSTOM")

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios
sharpe_ratios_custom = (df_all_returns.mean() * 252) / (df_all_returns.std() * np.sqrt(252))
sharpe_ratios_custom

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios_custom.plot.bar(figsize=(20,10), title="Sharpe Ratios")

### How does your portfolio do?

Write your answer here!

In [ ]:
#Note: For custom portfolio I understood the comment "at last one year" as at LEAST one year
#so used google finance to get data for MSFT, AAPL and AMZN from 01 JAN 2019 till date
#=GOOGLEFINANCE("MSFT", "price", "1/1/2019", "12/31/2021", "DAILY") and saved to .csv similar to OTEX

##The perfomance and risk analysis done above shows that my custom portfolio is ??
## The custom portfolio as per my risk analysis falls in the highest risk based on
## Annalized standard deviation
## My custom portfolio has highest positive correlation with Berkshire Hathway and
## highest negative correlation with Algo 1
## Custom Portfolio has a good sharpe ration 3.5 but there are other stocks outperforming
##it using this measurement


